## PREPROCESSING DATA

In [1]:
from preprocessing import Parser, ReportPreprocessor, SrcPreprocessor
from datasets import DATASET

from collections import defaultdict, OrderedDict

In [2]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\nhatm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\nhatm\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [3]:
parser = Parser(DATASET)
bug_reports = parser.report_parser()
src_files = parser.src_parser()

In [4]:
src_processor = SrcPreprocessor(src_files, parser.name).preprocess()
report_processor = ReportPreprocessor(bug_reports, parser.name).preprocess()

## INDEXING SOURCE CODE FILES


In [6]:
import pandas as pd

In [7]:
file_path = "outputs/tomcat_source_code_data.csv"
src_df = pd.read_csv(file_path)

In [8]:
def prepare_data(key_string, value_string):
    data = {}
    for index, row in src_df.iterrows():
        all_content = eval(row[value_string])
        data[row[key_string]] = all_content['stemmed']
    return data

In [9]:
def build_inverted_index(documents):
    inverted_index = defaultdict(list)
    for key, contents in documents.items():
        for content in contents:
            inverted_index[content].append(key)
    return inverted_index

In [18]:
data_src_codes_content = prepare_data('key','all_content')
data_src_codes_size = src_df[['key', 'total_lines']].values.tolist()
inverted_index_source_codes = build_inverted_index(data_src_codes_content)

## BUILD VSM

In [21]:
import math

In [22]:
def compute_document_tf_idf(documents, inverted_index):
    tf_idf = defaultdict(lambda: defaultdict(float))
    total_documents = len(documents)

    for key, contents in documents.items():
        content_frequency = defaultdict(int)
        for content in contents:
            content_frequency[content] += 1

        for content, count in content_frequency.items():
            tf = 1 + math.log(count, 10) if count > 0 else 0
            df = len(inverted_index[content]) if content in inverted_index else 0
            idf = math.log(total_documents / (1 + df), 10)

            tf_idf[key][content] = tf * idf

    for doc_id in tf_idf:
        norm = math.sqrt(sum(weight ** 2 for weight in tf_idf[doc_id].values()))
        if norm > 0:
            for content in tf_idf[doc_id]:
                tf_idf[doc_id][content] /= norm

    return tf_idf

In [23]:
vsm = compute_document_tf_idf(data_src_codes_content, inverted_index_source_codes)

In [24]:
vsm_iter = list(vsm.items())

## CALCULATE COEFFICIENT FILE SIZE

In [40]:
max_size = 0
min_size = math.inf
for data_size in data_src_codes_size:
    if data_size[1] > max_size:
        max_size = data_size[1]
    if data_size[1] < min_size:
        min_size = data_size[1]

In [42]:
def calculate_coefficient_size(size):
    value_standardization = (size - min_size) / (max_size - min_size)
    return 1/(1 + math.exp(-value_standardization))

In [47]:
coefficient_size = defaultdict(float)
for data_size in data_src_codes_size:
    coefficient_size[data_size[0]] = calculate_coefficient_size(data_size[1])

## HANDLE REPORT DATA

In [ ]:
def prepare_data(key_string, value_string):
    data = {}
    for index, row in src_df.iterrows():
        all_content = eval(row[value_string])
        data[row[key_string]] = all_content['stemmed']
    return data

## RANKING RESULTS